In [1]:
import urllib.request
import pandas as pd
from pandas import DataFrame
import re
from selenium.webdriver import Chrome
import time
import numpy as np

In [2]:
webdriver = "/Users/jacks/Selenium/chromedriver"

sorter = "abcdefghijklmnopqrstuvwxyz"

indices = []
prices_3_6_2020 = []
prices_1_6_2020 = []
symbols = []
companies = []

In [6]:
symbols = []
companies = []
for letter in sorter:
    url = "https://markets.businessinsider.com/index/components/s&p_500/" + letter + "/"
    stream = urllib.request.urlopen(url)
    for line in stream:
        decoded = line.decode('UTF-8').strip()
        finder = re.compile(r"\"/stocks/(\w{1,5})-stock\" title=\"(\D+)\">")
        if finder.search(str(line)):
            company = finder.search(str(line)).group(2)
            symbol = finder.search(str(line)).group(1)
            if company not in companies:
                companies.append(str(company))
                symbols.append(str(symbol).upper())
    stream.close()
table = DataFrame({'Company Name': companies,
                   'Symbol': symbols})
table.to_csv("stocks.csv")
print(table)

                             Company Name Symbol
0                        A.O. Smith Corp.    AOS
1                     Abbott Laboratories    ABT
2                              AbbVie Inc   ABBV
3                           Accenture plc    ACN
4                Activision Blizzard Inc.   ATVI
5                      Acuity Brands Inc.    AYI
6                              Adobe Inc.   ADBE
7                 Advance Auto Parts Inc.    AAP
8                               AES Corp.    AES
9          Affiliated Managers Group Inc.    AMG
10                              Aflac Inc    AFL
11              Agilent Technologies Inc.      A
12        Air Products and Chemicals Inc.    APD
13                            Akamai Inc.   AKAM
14                  Alaska Air Group Inc.    ALK
15                        Albemarle Corp.    ALB
16                             Alcoa Corp     AA
17   Alexandria Real Estate Equities Inc.    ARE
18           Alexion Pharmaceuticals Inc.   ALXN
19                  

In [7]:
def index(begin, end):
    driver = Chrome(webdriver)
    for i in range(begin, end):
        url = "https://markets.businessinsider.com/stocks/" + symbols[i] + "-stock/"
        driver.get(url)
        time.sleep(4)
        exchanges = driver.find_elements_by_css_selector(".exchange.hidden-xs")
        if len(exchanges) < 1:
            del companies[i]
            del symbols[i]
        else:
            exchange = exchanges[0]
            indices.append(exchange.text)
        print(i)
    driver.close()

In [10]:
for i in range(0, len(symbols), 1):
    index(i, i+1)

442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479


IndexError: list index out of range

In [14]:
table = DataFrame({'Company Name': companies,
                   'Symbol': symbols,
                   'Stock Exchange': indices})

print(table)
table.to_csv("stocks.csv")

                             Company Name Symbol Stock Exchange
0                        A.O. Smith Corp.    AOS           NYSE
1                     Abbott Laboratories    ABT           NYSE
2                              AbbVie Inc   ABBV           NYSE
3                           Accenture plc    ACN           NYSE
4                Activision Blizzard Inc.   ATVI            NAS
5                      Acuity Brands Inc.    AYI           NYSE
6                              Adobe Inc.   ADBE            NAS
7                 Advance Auto Parts Inc.    AAP           NYSE
8                               AES Corp.    AES           NYSE
9          Affiliated Managers Group Inc.    AMG           NYSE
10                              Aflac Inc    AFL           NYSE
11              Agilent Technologies Inc.      A           NYSE
12        Air Products and Chemicals Inc.    APD           NYSE
13                            Akamai Inc.   AKAM            NAS
14                  Alaska Air Group Inc

In [29]:
def get_prices(begin, end):
    driver = Chrome(webdriver)
    for i in range(begin, end):
        exchange = indices[i]
        symbol = symbols[i]
        url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/" + exchange + "/6.1.2020_6.3.2020"
        driver.get(url)
        time.sleep(4)
        check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[2]/td[2]")
        if len(check_price) > 0:
            price = check_price[0].text
            prices_3_6_2020.append(price)
        else:
            if exchange != 'NAS' and exchange != 'NYSE':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NYSE/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[2]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_3_6_2020.append(price)
                    indices[i] = 'NYSE'
                else:
                    url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NAS/6.1.2020_6.3.2020"
                    driver.get(url)
                    time.sleep(4)
                    check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[2]/td[2]")
                    if len(check_price) > 0:
                        price = check_price[0].text
                        prices_3_6_2020.append(price)
                        indices[i] = 'NAS'
                    else:
                        indices[i] = 'Weird one'
                        prices_3_6_2020.append('--')
            elif exchange == 'NYSE':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NAS/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[2]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_3_6_2020.append(price)
                    indices[i] = 'NAS'
                else:
                    prices_3_6_2020.append("--")
            elif exchange == 'NAS':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NYSE/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[2]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_3_6_2020.append(price)
                    indices[i] = 'NYSE'
                else:
                    prices_3_6_2020.append("--")

        check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[44]/td[2]")
        if len(check_price) > 0:
            price = check_price[0].text
            prices_1_6_2020.append(price)
        else:
            if exchange != 'NAS' and exchange != 'NYSE':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NYSE/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[44]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_1_6_2020.append(price)
                    indices[i] = 'NYSE'
                else:
                    url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NAS/6.1.2020_6.3.2020"
                    driver.get(url)
                    time.sleep(4)
                    check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[44]/td[2]")
                    if len(check_price) > 0:
                        price = check_price[0].text
                        prices_1_6_2020.append(price)
                        indices[i] = 'NAS'
                    else:
                        indices[i] = 'Weird one'
                        prices_3_6_2020.append('--')
            elif exchange == 'NYSE':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NAS/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[44]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_1_6_2020.append(price)
                    indices[i] = 'NAS'
                else:
                    prices_1_6_2020.append("--")
            elif exchange == 'NAS':
                url = "https://markets.businessinsider.com/stocks/" + symbol + "/price/NYSE/6.1.2020_6.3.2020"
                driver.get(url)
                time.sleep(4)
                check_price = driver.find_elements_by_xpath("/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[14]/div/div[2]/table/tbody/tr[44]/td[2]")
                if len(check_price) > 0:
                    price = check_price[0].text
                    prices_1_6_2020.append(price)
                    indices[i] = 'NYSE'
                else:
                    prices_1_6_2020.append("--")
        print(i)
    driver.close()

In [36]:
for i in range(0, table.shape[0], 1):
    get_prices(i, i+1)

471
472
473
474
475
476
477
478
479


IndexError: list index out of range

In [55]:
table = DataFrame({'Company Name': companies,
                   'Symbol': symbols,
                   'Stock Exchange': indices,
                   'Price 3/6/2020': prices_3_6_2020,
                   'Price 1/6/2020': prices_1_6_2020})

print(table)
table.to_csv("stocks.csv")

                             Company Name Symbol Stock Exchange  \
0                        A.O. Smith Corp.    AOS           NYSE   
1                     Abbott Laboratories    ABT           NYSE   
2                              AbbVie Inc   ABBV           NYSE   
3                           Accenture plc    ACN           NYSE   
4                Activision Blizzard Inc.   ATVI            NAS   
5                      Acuity Brands Inc.    AYI           NYSE   
6                              Adobe Inc.   ADBE            NAS   
7                 Advance Auto Parts Inc.    AAP           NYSE   
8                               AES Corp.    AES           NYSE   
9          Affiliated Managers Group Inc.    AMG           NYSE   
10                              Aflac Inc    AFL           NYSE   
11              Agilent Technologies Inc.      A           NYSE   
12        Air Products and Chemicals Inc.    APD           NYSE   
13                            Akamai Inc.   AKAM            NA

In [7]:
price_change = []
for i in range(table.shape[0]):
    delta_price = float(table.at[i, 'Price 3/6/2020'].replace(',', '')) - float(table.at[i, 'Price 1/6/2020'].replace(',', ''))
    price_change.append(round(delta_price, 2))

table['Price Change'] = price_change
    

In [4]:
percent_change = []
for p in prices_3_6_2020:
    new_price = float(p.replace(',', ''))
    i = prices_3_6_2020.index(p)
    old_price = float(prices_1_6_2020[i].replace(',', ''))
    val = ((new_price - old_price) / old_price) * 100
    percent_change.append(str(round(val, 2)) + '%')
print(percent_change)

table['Percent Price Change'] = percent_change
print(table)
table.to_csv('stocks.csv')

['-14.09%', '-8.33%', '-0.01%', '-14.72%', '2.02%', '-29.53%', '3.7%', '-21.51%', '-18.53%', '-17.34%', '-25.36%', '-7.2%', '0.46%', '3.61%', '-37.19%', '16.0%', '-47.12%', '-1.08%', '-13.85%', '-20.57%', '-2.18%', '-0.88%', '-35.18%', '4.13%', '-7.24%', '-5.81%', '-5.43%', '-15.64%', '0.86%', '3.19%', '10.16%', '-44.83%', '2.66%', '-14.4%', '-23.41%', '6.43%', '12.41%', '-22.98%', '0.73%', '-14.26%', '-14.01%', '-15.9%', '-9.68%', '-6.21%', '-5.94%', '1.09%', '-10.22%', '-4.0%', '-4.36%', '-19.12%', '-19.31%', '3.89%', '-12.19%', '-5.8%', '-3.24%', '-9.26%', '-13.58%', '-0.28%', '-10.09%', '-36.47%', '15.62%', '-25.89%', '-25.34%', '-0.25%', '-12.07%', '-12.86%', '6.21%', '-9.64%', '-23.01%', '-30.95%', '-2.85%', '-19.42%', '-21.27%', '-6.39%', '-14.09%', '-16.17%', '-16.57%', '-7.55%', '6.95%', '-19.69%', '-39.88%', '3.25%', '-2.9%', '-45.47%', '-19.2%', '-5.4%', '-9.86%', '-6.34%', '-17.82%', '-4.07%', '-3.97%', '-18.69%', '-29.35%', '-1.18%', '-76.84%', '-22.48%', '-18.84%', '-9.95

In [32]:
pes = []
bookv = []
debt_rat = []
cash_flow = []
future_flow = []
def get_more_stats(begin, end):
    driver = Chrome(webdriver)
    for i in range(begin, end):
        symbol = symbols[i]
        url = "https://markets.businessinsider.com/stocks/" + symbol + "/financials"
        driver.get(url)
        time.sleep(4)
        arr = driver.find_elements_by_xpath('/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[13]/div/table/tbody/tr[6]/td[2]')
        if len(arr) > 0:
            pes.append(arr[0].text)
        else:
            pes.append("--")
        arr = driver.find_elements_by_xpath('/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[25]/div[2]/table/tbody/tr[8]/td[8]')
        if len(arr) > 0:
            debt_rat.append(arr[0].text)
        else:
            debt_rat.append("--")
        arr = driver.find_elements_by_xpath('/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[13]/div/table/tbody/tr[20]/td[2]')
        if len(arr) > 0:
            cash_flow.append(arr[0].text)
        else:
            cash_flow.append("--")
        arr = driver.find_elements_by_xpath('/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[13]/div/table/tbody/tr[20]/td[3]')
        if len(arr) > 0:
            future_flow.append(arr[0].text)
        else:
            future_flow.append("--")
        arr = driver.find_elements_by_xpath('/html/body/div[2]/div[6]/div[2]/div/div[2]/div[3]/div/div[13]/div/table/tbody/tr[22]/td[2]')
        if len(arr) > 0:
            bookv.append(arr[0].text)
        else:
            bookv.append("--")
        print(i)
        driver.close()

In [33]:
for i in range(0, table.shape[0], 1):
    get_more_stats(i, i+1)
print(pes)
print(cash_flow)
print(bookv)
print(debt_rat)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [34]:
table['P/E'] = pes
table['Cash Flow'] = cash_flow
table['Book Value per Share'] = bookv
table['Debt Ratio'] = debt_rat
table['Projected Cash Flow'] = future_flow
print(table)

                             Company Name Symbol Stock Exchange  \
0                        A.O. Smith Corp.    AOS           NYSE   
1                     Abbott Laboratories    ABT           NYSE   
2                              AbbVie Inc   ABBV           NYSE   
3                           Accenture plc    ACN           NYSE   
4                Activision Blizzard Inc.   ATVI            NAS   
5                      Acuity Brands Inc.    AYI           NYSE   
6                              Adobe Inc.   ADBE            NAS   
7                 Advance Auto Parts Inc.    AAP           NYSE   
8                               AES Corp.    AES           NYSE   
9          Affiliated Managers Group Inc.    AMG           NYSE   
10                              Aflac Inc    AFL           NYSE   
11              Agilent Technologies Inc.      A           NYSE   
12        Air Products and Chemicals Inc.    APD           NYSE   
13                            Akamai Inc.   AKAM            NA

In [7]:
non_complete = []
for i in range(table.shape[0]):
    if table.at[i, 'P/E'] == '-' or table.at[i, 'P/E'] == '--':
        if not i in non_complete: 
            non_complete.append(i)
    if table.at[i, 'Book Value per Share'] == '-' or table.at[i, 'Book Value per Share'] == '--':
        if not i in non_complete: 
            non_complete.append(i)
    if table.at[i, 'Debt Ratio'] == '-' or table.at[i, 'Debt Ratio'] == '--':
        if not i in non_complete: 
            non_complete.append(i)
    if table.at[i, 'Cash Flow'] == '-' or table.at[i, 'Cash Flow'] == '--':
        if not i in non_complete: 
            non_complete.append(i)
    if table.at[i, 'Projected Cash Flow'] == '-' or table.at[i, 'Cash Flow'] == '--':
        if not i in non_complete: 
            non_complete.append(i)
table = table.drop(table.index[non_complete])
table = table.reset_index()
print(table)

                           Company Name Symbol Stock Exchange Price 3/6/2020  \
0                      A.O. Smith Corp.    AOS           NYSE          40.49   
1                   Abbott Laboratories    ABT           NYSE          78.62   
2                            AbbVie Inc   ABBV           NYSE           88.2   
3                         Accenture plc    ACN           NYSE         177.45   
4              Activision Blizzard Inc.   ATVI            NAS          59.65   
6                            Adobe Inc.   ADBE            NAS         340.45   
7               Advance Auto Parts Inc.    AAP           NYSE         123.81   
8                             AES Corp.    AES           NYSE          16.09   
11            Agilent Technologies Inc.      A           NYSE          77.95   
12      Air Products and Chemicals Inc.    APD           NYSE         226.19   
13                          Akamai Inc.   AKAM            NAS          89.78   
14                Alaska Air Group Inc. 

In [26]:
pe_arr = []
cash_flow_diff_arr = []
dr_arr = []
ppc_arr = []
bv_arr = []

for i in range(table.shape[0]):
    pe = float(table.at[i, 'P/E'])
    pe_arr.append(pe)
    dr = float(table.at[i, 'Debt Ratio'])
    dr_arr.append(dr)
    cf = float(table.at[i, 'Cash Flow'].replace(',', ''))
    pro_cf = float(table.at[i, 'Projected Cash Flow'].replace(',', ''))
    cash_flow_diff_arr.append(((pro_cf - cf) / cf) * 100)
    ppc = float(table.at[i, 'Percent Price Change'].replace('%', ''))
    ppc_arr.append(ppc)
    bv = float(table.at[i, 'Book Value per Share'])
    bv_arr.append(bv)
    
pe_arr.sort(reverse=True)
cash_flow_diff_arr.sort()
dr_arr.sort(reverse=True)
ppc_arr.sort(reverse=True)
bv_arr.sort(reverse=True)

In [28]:
rating_arr = []
for i in range(table.shape[0]):
    pe = float(table.at[i, 'P/E'])
    rating1 = ((pe_arr.index(pe) + 1) / len(pe_arr)) * 10
    dr = float(table.at[i, 'Debt Ratio'])
    rating2 = ((dr_arr.index(dr) + 1) / len(dr_arr)) * 10
    cf = float(table.at[i, 'Cash Flow'].replace(',', ''))
    pro_cf = float(table.at[i, 'Projected Cash Flow'].replace(',', ''))
    cf_diff = ((pro_cf - cf) / cf) * 100
    rating3 = ((cash_flow_diff_arr.index(cf_diff) + 1) / len(cash_flow_diff_arr)) * 10
    bv = float(table.at[i, 'Book Value per Share'])
    rating4 = ((bv_arr.index(bv) + 1) / len(bv_arr)) * 10
    ppc = float(table.at[i, 'Percent Price Change'].replace('%', ''))
    rating5 = ((ppc_arr.index(ppc) + 1) / len(ppc_arr)) * 10
    rating = (rating1 + rating2 + rating3 + rating4 + rating5) / 5
    rating_arr.append(round(rating, 2))
print(rating_arr)
table["Investability Rating"] = rating_arr
table.to_csv('stocks.csv')
table

[5.51, 4.81, 5.13, 5.36, 4.97, 4.08, 4.37, 6.1, 6.19, 3.04, 4.87, 6.45, 3.08, 7.45, 6.63, 6.25, 3.83, 5.33, 5.64, 3.29, 4.73, 4.68, 6.89, 2.77, 5.5, 1.93, 7.57, 2.25, 3.47, 1.61, 3.9, 5.33, 5.33, 5.54, 5.08, 4.89, 4.17, 3.7, 3.28, 3.83, 5.46, 5.88, 6.07, 4.63, 4.15, 4.94, 4.98, 3.48, 4.62, 7.86, 3.75, 4.58, 4.26, 4.76, 4.5, 3.2, 6.12, 7.21, 6.65, 5.16, 5.22, 4.74, 7.0, 5.32, 4.65, 7.66, 3.99, 4.42, 6.12, 5.46, 4.9, 5.92, 4.94, 6.69, 4.88, 5.34, 5.44, 5.74, 2.26, 8.46, 5.78, 4.03, 4.17, 7.85, 4.17, 6.48, 4.42, 2.74, 2.33, 2.21, 4.7, 5.65, 3.4, 5.28, 6.21, 7.42, 6.53, 4.42, 4.64, 4.09, 7.52, 3.19, 7.38, 1.95, 4.53, 4.81, 4.88, 5.04, 4.29, 5.63, 4.76, 4.67, 5.2, 7.17, 6.6, 2.17, 6.17, 6.15, 4.15, 4.13, 5.28, 4.49, 4.83, 4.66, 3.1, 2.06, 6.85, 5.85, 5.53, 4.17, 5.38, 2.72, 3.0, 5.28, 4.29, 4.19, 5.83, 1.64, 6.75, 4.29, 5.11, 3.42, 4.29, 3.86, 5.08, 5.5, 5.71, 3.19, 7.16, 5.96, 5.79, 3.95, 3.97, 4.34, 4.98, 4.74, 7.35, 6.9, 6.29, 4.06, 7.22, 7.51, 5.36, 5.22, 5.8, 6.99, 5.07, 4.14, 4.15, 6.

,Company Name,Symbol,Stock Exchange,Price 3/6/2020,Price 1/6/2020,Price Change,P/E,Cash Flow,Book Value per Share,Debt Ratio,Percent Price Change,Projected Cash Flow,Investability Rating
0,A.O. Smith Corp.,AOS,NYSE,40.49,47.13,-6.64,16.26,355,14.40,45.49,-14.09%,297,5.51
1,Abbott Laboratories,ABT,NYSE,78.62,85.76,-7.14,21.90,"6,422",24.82,53.89,-8.33%,"7,144",4.81
2,AbbVie Inc,ABBV,NYSE,88.2,88.21,-0.01,8.60,"22,106",-0.21,109.17,-0.01%,"24,245",5.13
3,Accenture plc,ACN,NYSE,177.45,208.09,-30.64,21.40,"6,039",24.81,50.23,-14.72%,"6,703",5.36
4,Activision Blizzard Inc.,ATVI,NAS,59.65,58.47,1.18,23.84,"1,962",18.07,35.47,2.02%,"2,201",4.97
5,Adobe Inc.,ADBE,NAS,340.45,328.29,12.16,32.16,"5,197",33.74,50.35,3.7%,"6,112",4.08
6,Advance Auto Parts Inc.,AAP,NYSE,123.81,157.74,-33.93,16.16,687,56.26,68.45,-21.51%,726,4.37
7,AES Corp.,AES,NYSE,16.09,19.75,-3.66,11.06,974,6.01,81.82,-18.53%,"1,082",6.10
8,Agilent Technologies Inc.,A,NYSE,77.95,84,-6.05,20.37,752,16.74,49.77,-7.2%,"1,110",6.19
9,Air Products and Chemicals Inc.,APD,NYSE,226.19,225.15,1.04,22.67,-381,56.55,39.88,0.46%,556,3.04
